<a href="https://colab.research.google.com/github/usernamedmanas/My-Projects/blob/main/News_Recommender_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataframes
articles_df = pd.read_csv('/content/drive/MyDrive/CNN_Articels_clean.csv')
user_df = pd.read_csv('/content/drive/MyDrive/user - user.csv')
articles_df = articles_df.iloc[:,:11]

# Step 1: Choose a user
user_id = 2

# Step 2: Extract user interactions
user_data = user_df[user_df['userId'] == user_id].iloc[0]
read_articles = [int(article_id) for article_id in user_data['Articles-Read'].split(',')]
liked_articles = [int(article_id) for article_id in user_data['Articles-Liked'].split(',')]
commented_articles = [int(article_id) for article_id in user_data['Article-commented'].split(',')]

# Step 3: Filter out user-interacted articles
filtered_articles_df = articles_df[~articles_df['Index'].isin(read_articles + liked_articles + commented_articles)].copy()

# Step 4: Combine text features
filtered_articles_df['CombinedFeatures'] = filtered_articles_df['Description'].fillna('') + ' ' + filtered_articles_df['Author'].fillna('') + ' ' + filtered_articles_df['Category'].fillna('') + ' ' + filtered_articles_df['Section'].fillna('')

# Step 5: Handle missing values
filtered_articles_df['CombinedFeatures'].fillna('', inplace=True)

# Step 6: Calculate similarity
vectorizer = TfidfVectorizer()
articles_tfidf = vectorizer.fit_transform(filtered_articles_df['CombinedFeatures'])
user_vector = vectorizer.transform([user_data['Search']])
similarity_scores = cosine_similarity(articles_tfidf, user_vector)

# Step 7: Assign weightage to similarity scores
weightage = {'search': 1, 'read': 1, 'liked': 1, 'commented': 1}
weighted_scores = similarity_scores * weightage['search']
weighted_scores[read_articles] *= weightage['read']
weighted_scores[liked_articles] *= weightage['liked']
weighted_scores[commented_articles] *= weightage['commented']

# Step 8: Sort articles based on weighted similarity scores
recommendations = filtered_articles_df.assign(WeightedScore=weighted_scores.flatten()).sort_values(by='WeightedScore', ascending=False)

# Step 9: Select top 5 recommendations
top_recommendations = recommendations.head(5)

# Display the recommendations
print(top_recommendations.drop(['CombinedFeatures','WeightedScore'],axis=1))


<ipython-input-7-0c1d74398679>:6: DtypeWarning: Columns (0,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,26

     Index                       Author    Date published  Category Section  \
16      20  Rishi Iyengar, CNN Business  11-03-2022 18:41  business    tech   
157    203   Michelle Toh, CNN Business  03-12-2020 05:47  business    tech   
2450  3067             Jill Martin, CNN  15-07-2021 22:27     sport  tennis   
386    713   Shannon Liao, CNN Business  22-02-2020 18:28  business    tech   
392    725   Shannon Liao, CNN Business  03-04-2020 17:54  business    tech   

                                                    Url  \
16    https://www.cnn.com/2022/03/11/tech/russia-int...   
157   https://www.cnn.com/2020/12/03/tech/autox-robo...   
2450  https://www.cnn.com/2021/07/15/tennis/novak-dj...   
386   https://www.cnn.com/2020/02/22/tech/lowkey-gg-...   
392   https://www.cnn.com/2020/04/03/tech/esports-sp...   

                                               Headline  \
16    Why internet backbone services Cogent and Lume...   
157   Self-driving robotaxis are taking off in China